In [1]:
import pymysql
import requests

### 下載分析pm2.5 opendata資料

In [5]:
url="https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON"
url

'https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON'

In [6]:
resp=requests.get(url,verify=False)
resp

c:\Users\USER\Desktop\django\pm25-project\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.moenv.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [7]:
datas=resp.json()['records']
datas

[{'site': '員林',
  'county': '彰化縣',
  'pm25': '14',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '大城',
  'county': '彰化縣',
  'pm25': '',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '富貴角',
  'county': '新北市',
  'pm25': '4',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '麥寮',
  'county': '雲林縣',
  'pm25': '11',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '關山',
  'county': '臺東縣',
  'pm25': '3',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '馬公',
  'county': '澎湖縣',
  'pm25': '7',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '金門',
  'county': '金門縣',
  'pm25': '6',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '馬祖',
  'county': '連江縣',
  'pm25': '',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '埔里',
  'county': '南投縣',
  'pm25': '10',
  'datacreationdate'

In [12]:
len(datas)

1000

In [13]:
datas[0]

{'site': '員林',
 'county': '彰化縣',
 'pm25': '4',
 'datacreationdate': '2025-08-20 19:00',
 'itemunit': 'μg/m3'}

### 建立資料表

In [3]:
table_str="""
create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)
"""
print(table_str)


create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)



### 連接資料庫

In [3]:
conn=pymysql.connect(
host="mysql-39175e7e-mdream-81ff.i.aivencloud.com",
user="avnadmin",
password="AVNS_VUfo1XoxZwEp-i2sOw8",
port=18633,
database="defaultdb"
)

cursor=conn.cursor()
cursor


In [4]:
cursor.execute(table_str)
conn.commit()

### 寫入一筆資料

In [30]:
sqlstr="insert into pm25(site,county,pm25,datacreationdate,itemunit)\
      values('{}','{}',{},'{}','{}')"

data=list(datas[0].values())
print(data)
sqlstr.format(data[0],data[1],data[2],data[3],data[4])

['員林', '彰化縣', '4', '2025-08-20 19:00', 'μg/m3']


"insert into pm25(site,county,pm25,datacreationdate,itemunit)      values('員林','彰化縣',4,'2025-08-20 19:00','μg/m3')"

In [31]:
cursor.execute(sqlstr.format(data[0],data[1],data[2],data[3],data[4]))
conn.commit()

### 一次寫入  
- 加上 ignore

In [8]:
sqlstr="insert ignore into pm25(site,county,pm25,datacreationdate,itemunit)\
      values(%s,%s,%s,%s,%s)"

# 移除pm25空字串部分
values=[list(data.values()) for data in datas if list(data.values())[2]!=""]

cursor.executemany(sqlstr,values)
conn.commit()

In [9]:
conn.close()

### 取得各縣市站點的平均值

In [ ]:
cursor.execute("select avg(pm25) from pm25;")
float(cursor.fetchone()[0])

In [18]:
sqlstr="""
select county,round(avg(pm25),2)  from pm25 group by county;
"""

cursor.execute(sqlstr)
cursor.fetchall()

(('彰化縣', Decimal('12.49')),
 ('新北市', Decimal('3.52')),
 ('雲林縣', Decimal('10.58')),
 ('臺東縣', Decimal('2.42')),
 ('澎湖縣', Decimal('7.37')),
 ('金門縣', Decimal('10.32')),
 ('南投縣', Decimal('9.93')),
 ('高雄市', Decimal('12.31')),
 ('桃園市', Decimal('6.23')),
 ('宜蘭縣', Decimal('3.02')),
 ('臺北市', Decimal('3.83')),
 ('花蓮縣', Decimal('3.18')),
 ('屏東縣', Decimal('10.25')),
 ('臺南市', Decimal('10.04')),
 ('嘉義市', Decimal('8.13')),
 ('嘉義縣', Decimal('9.99')),
 ('臺中市', Decimal('12.15')),
 ('苗栗縣', Decimal('9.85')),
 ('新竹市', Decimal('9.56')),
 ('新竹縣', Decimal('7.93')),
 ('基隆市', Decimal('2.38')),
 ('連江縣', Decimal('6.50')))

In [19]:
conn.close()

### 取的單一縣市的站點數值

In [ ]:
county="新北市"

# 取得最新資料
sqlstr='''
select site,pm25,datacreationdate from pm25
where county=%s;
'''

cursor.execute(sqlstr,(county,))
datas=cursor.fetchall()
datas

(('富貴角', 4, datetime.datetime(2025, 8, 27, 20, 0)),
 ('永和', 3, datetime.datetime(2025, 8, 27, 20, 0)),
 ('三重', 7, datetime.datetime(2025, 8, 27, 20, 0)),
 ('淡水', 7, datetime.datetime(2025, 8, 27, 20, 0)),
 ('林口', 5, datetime.datetime(2025, 8, 27, 20, 0)),
 ('菜寮', 5, datetime.datetime(2025, 8, 27, 20, 0)),
 ('新莊', 3, datetime.datetime(2025, 8, 27, 20, 0)),
 ('板橋', 3, datetime.datetime(2025, 8, 27, 20, 0)),
 ('土城', 3, datetime.datetime(2025, 8, 27, 20, 0)),
 ('新店', 2, datetime.datetime(2025, 8, 27, 20, 0)),
 ('汐止', 6, datetime.datetime(2025, 8, 27, 20, 0)),
 ('富貴角', 4, datetime.datetime(2025, 8, 27, 19, 0)),
 ('永和', 6, datetime.datetime(2025, 8, 27, 19, 0)),
 ('三重', 10, datetime.datetime(2025, 8, 27, 19, 0)),
 ('淡水', 12, datetime.datetime(2025, 8, 27, 19, 0)),
 ('林口', 6, datetime.datetime(2025, 8, 27, 19, 0)),
 ('菜寮', 5, datetime.datetime(2025, 8, 27, 19, 0)),
 ('新莊', 7, datetime.datetime(2025, 8, 27, 19, 0)),
 ('板橋', 3, datetime.datetime(2025, 8, 27, 19, 0)),
 ('土城', 6, datetime.datetim